In [5]:
import pandas as pd


df = pd.read_csv('Data/Ridership.csv')
df.sample(5)

,Year,Month,Day,Week Number,Corridor,Workday,Station,Period,Ridership,N_trains,Covid19
48442,2022,January,6,1,Corridor_4,y,Station_3,Midday,108,6,1
5865,2019,April,26,17,Corridor_2,y,Station_14,AM Peak,720,1,0
9020,2019,July,4,27,Corridor_4,y,Station_3,AM Peak,45,1,0
26519,2020,July,24,30,Corridor_5,y,Station_19,AM Peak,130,3,1
8330,2019,June,19,25,Corridor_1,y,Station_3,Midday,407,6,0


In [6]:
months = {'January': 1, 'February': 2, 'March': 3, 'April': 4,
          'May': 5, 'June': 6, 'July': 7, 'August': 8,
          'September': 9, 'October': 10, 'November': 11, 'December': 12}

df['month'] = df['Month'].map(months)

df['date'] = pd.to_datetime(df[['Year', 'month', 'Day']])
df.head()

,Year,Month,Day,Week Number,Corridor,Workday,Station,Period,Ridership,N_trains,Covid19,month,date
0,2019,January,1,1,Corridor_1,n,Station_1,Weekend/Holiday,174,3,0,1,2019-01-01
1,2019,January,1,1,Corridor_1,n,Station_2,Weekend/Holiday,378,10,0,1,2019-01-01
2,2019,January,1,1,Corridor_1,n,Station_3,Weekend/Holiday,599,12,0,1,2019-01-01
3,2019,January,1,1,Corridor_2,n,Station_4,Weekend/Holiday,2759,35,0,1,2019-01-01
4,2019,January,1,1,Corridor_2,n,Station_5,Weekend/Holiday,2629,36,0,1,2019-01-01


In [9]:
df.to_parquet('Data/Ridership.parquet', index=False)

In [7]:
df.to_parquet('Data/parquetdata/', index=False, engine='pyarrow', compression='snappy', partition_cols=['Year', 'Month'])

In [10]:
pd.read_parquet('Data/Ridership.parquet', engine='pyarrow', columns=['Year', 'Month']).head()

,Year,Month
0,2019,January
1,2019,January
2,2019,January
3,2019,January
4,2019,January


In [11]:
a = []
b = [1, 2, 3]

a.append(b)

a

[[1, 2, 3]]

In [12]:
a.extend(b)

a

[[1, 2, 3], 1, 2, 3]

In [14]:
from glob import glob
file_path = glob(f'Data/parquetdata/Year=2019/Month=August/*.parquet')
df = pd.read_parquet(file_path[0], engine='pyarrow')
df.sample(5)

,Day,Week Number,Corridor,Workday,Station,Period,Ridership,N_trains,Covid19,month,date
237,7,32,Corridor_4,y,Station_9,Midday,644,1,0,8,2019-08-07
345,8,32,Corridor_7,y,Station_24,PM Peak,120,2,0,8,2019-08-08
1084,23,34,Corridor_7,y,Station_3,PM Peak,3562,5,0,8,2019-08-23
49,1,31,Corridor_5,y,Station_19,AM Peak,7411,8,0,8,2019-08-01
1144,26,35,Corridor_3,y,Station_13,Midday,998,6,0,8,2019-08-26


In [15]:
from glob import glob

file_paths = []
for month in ['August', 'April']:
    file_paths.extend(glob(f'Data/parquetdata/Year=2019/Month={month}/*.parquet'))


file_paths

['Data/parquetdata/Year=2019/Month=August/01402f55654647fbbbef9640eb105289-0.parquet',
 'Data/parquetdata/Year=2019/Month=April/01402f55654647fbbbef9640eb105289-0.parquet']

In [17]:
dfs = []

for file_path in file_paths:
    df = pd.read_parquet(file_path, engine='pyarrow', columns=['date', 'Period', 'Ridership'])
    dfs.append(df)

partial_df1 = pd.concat(dfs, ignore_index=True)
partial_df1.sample(10)

,date,Period,Ridership
882,2019-08-20,Evening,1246
2143,2019-04-15,AM Peak,1055
1447,2019-04-01,PM Peak,5263
1090,2019-08-24,Weekend/Holiday,25587
1561,2019-04-02,PM Peak,4654
2212,2019-04-16,AM Peak,8198
2296,2019-04-18,Midday,579
54,2019-08-01,Midday,771
1456,2019-04-01,Evening,1764
434,2019-08-12,Evening,61


In [18]:
df.columns.to_list()

['date', 'Period', 'Ridership']

In [19]:
dfs = []

for file_path in file_paths:
    df = pd.read_parquet(file_path, engine='pyarrow', columns=['Day', 'Week Number', 'Corridor', 'Station', 'date', 'Period'])
    dfs.append(df)

partial_df2 = pd.concat(dfs, ignore_index=True)
partial_df2.sample(10)

,Day,Week Number,Corridor,Station,date,Period
47,1,31,Corridor_3,Station_3,2019-08-01,PM Peak
2631,26,17,Corridor_1,Station_1,2019-04-26,Midday
1492,1,14,Corridor_6,Station_3,2019-04-01,Evening
1531,2,14,Corridor_2,Station_3,2019-04-02,PM Peak
2528,24,17,Corridor_2,Station_14,2019-04-24,AM Peak
888,20,34,Corridor_7,Station_24,2019-08-20,Midday
1907,10,15,Corridor_4,Station_9,2019-04-10,AM Peak
2404,22,17,Corridor_2,Station_3,2019-04-22,Evening
658,15,33,Corridor_3,Station_16,2019-08-15,Midday
1569,3,14,Corridor_1,Station_6,2019-04-03,AM Peak


In [20]:
final_df = partial_df1.merge(partial_df2, on=['date', 'Period'])
final_df.head()

,date,Period,Ridership,Day,Week Number,Corridor,Station
0,2019-08-01,AM Peak,4716,1,31,Corridor_1,Station_1
1,2019-08-01,AM Peak,4716,1,31,Corridor_1,Station_6
2,2019-08-01,AM Peak,4716,1,31,Corridor_4,Station_7
3,2019-08-01,AM Peak,4716,1,31,Corridor_4,Station_8
4,2019-08-01,AM Peak,4716,1,31,Corridor_4,Station_9


In [21]:
final_df['Ridership'].sum()

np.int64(104803526)

In [22]:
final_df[final_df['Corridor'] == 'Corridor_1']['Ridership'].sum()

np.int64(15864362)

In [ ]:
# my first change in new_branch